In [76]:
from dotenv import load_dotenv
import os
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [120]:
from langchain_groq import ChatGroq

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import langchain




In [85]:
model =ChatGroq(model='deepseek-r1-distill-llama-70b')
embeddings = OllamaEmbeddings(model="llama3")

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


LOAD PDF

In [79]:
urls = 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf'
loader = PyPDFLoader(urls)
pages =[]

async for page in loader.alazy_load():
    pages.append(page)

In [123]:
npages = loader.load_and_split()

In [124]:
npages

[Document(metadata={'source': 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf', 'page': 1, 'page_label': '2'}, page_content='COVENANT UNIVERSITY \nSTUDENT HANDBOOK \n \n \n \n \n \n(2019 – 2022)'),
 Document(metadata={'source': 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf', 'page': 2, 'page_label': '3'}, page_content='i \nCovenant University \nStudent Handbook | 2019 - 2022 \n \nThe Board of Regents of Covenant University reserves the right to alter any \nsection, provision, or clause of this document as it deems fit from t ime to \ntime. \n \n \n \n \n \n \n \n \n \nPublished by  \nCovenant University,  \nKM 10, Idiroko Road,  \nP.M.B. 1023, Ota, Ogun State, Nigeria'),
 Document(metadata={'source': 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf', 'page': 3, 'page_label': '4'}, page_content='ii \nCHANCELLOR \nDavid O. Oyedepo, Ph.D., FNAE \n \nPRINCIPAL OFFICERS \nVICE-CHANCELLOR Prof. AAA.

Split into chunk size 

In [82]:
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.'],
    chunk_size=1000,
    chunk_overlap=400)


In [91]:
pages[0]

Document(metadata={'source': 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf', 'page': 0, 'page_label': '1'}, page_content='')

In [96]:

docs = splitter.split_documents(pages)

CREATE EMBEDDINGS

In [105]:

data_embedding = FAISS.from_documents(docs, embeddings)

In [106]:
file_name ='vector_index.pkl'

with open(file_name, 'wb') as f:
    pickle.dump(data_embedding, f)

In [110]:
data_embedding

RETRIEVAL QA

In [111]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=model, retriever = data_embedding.as_retriever())

In [121]:
langchain.debug = True

In [122]:

chain.invoke({'question':'List all service unit I can join'},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "List all service unit I can join"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "through the following units: \ni. Contemporary Choir \nii. Classical Choir \niii. Ushering Unit \niv. Hospitality Unit \nv. Technical Unit \nvi. Reflections Dance Group \nvii. Attendance Unit \nviii. Evangelism Team \nix. Prayer Force \nx. Sanctuary Keepers Unit \nxi. Decoration Unit \nxii. Communion Stewards Unit \nxiii. Word Study Unit \nxiv. Covenant University Theatre Group  \nThe activities of each Unit are coordinated by each Unit Executives \nwho report to the University Chaplain.",
      "question": "List all service unit I can join"
    },
    {
      "co

{'answer': '<think>\nThe provided document does not contain a list of service units you can join. It only mentions that "recognized Chapel units" are permitted with written authorization from the Chaplaincy, while "Para-service Units are prohibited."\n\n',
 'sources': 'https://www.covenantuniversity.edu.ng/downloads/Student-handbook-Feb-2020.pdf'}

In [114]:
chain({'question':'Are students meant to wear skirt on saturday'})

{'question': 'Are students meant to wear skirt on saturday',
 'answer': '<think>\nAlright, let me try to figure out the answer to the ',
 'sources': '"Are students meant to wear skirts on Saturday?" I\'ll start by reading through the provided content carefully.'}

In [139]:
load_dotenv()

True